In [3]:
%matplotlib qt4
from __future__ import division

from models import tools, filters

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
data = tools.load_data(limit=3000000)
data = data[data['response_time'] < 15000]

In [4]:
data = data[filters.world_countries(data) & filters.open_questions(data)]

In [6]:
_, bins = pd.cut(data['response_time'], bins=20, retbins=True)
intervals = zip(bins[:-1], bins[1:])

responses = defaultdict(lambda: [])
for lower_bound, upper_bound in intervals:
    tools.echo('{}-{}'.format(lower_bound, upper_bound))
    for place in data['place_id'].unique():
        vals = data[(data['response_time'] >= lower_bound) &
                    (data['response_time'] < upper_bound) &
                    (data['place_id'] == place)]
        responses[place].append(vals['is_correct'].mean())

14249.85-14999.0


In [7]:
X = [[] for _ in intervals]
for place in responses:
    for i, value in enumerate(responses[place]):
        if np.isfinite(value):
            X[i].append(value)

In [9]:
labels = ['({}, {}]'.format(int(i), int(j)) for i, j in intervals]
plt.xticks(rotation=70)
plt.boxplot(X, labels=labels, showfliers=False)
plt.xlabel('response time')
plt.ylabel('probability of recall')
plt.subplots_adjust(bottom=0.25)